In [1]:
import os

import numpy as np
import pandas as pd
import torch
from torch import Tensor

from train import ModelManager, DIR_ROOT_SAVE, DIR_ROOT_LOG
from utils import get_loader, get_cifar10

# globals
NUM_CLASSES = 10
BATCH_SIZE = 25
NUM_TRAIN = 50000
NUM_TEST = 10000
EPOCHS = 120

NOTEBOOK_NAME = 'tuning_cifar10'
PATH_EL2N = os.path.join(DIR_ROOT_SAVE, 'el2n_resnet18_cifar10', 'general.pt')
PATH_FORGET = os.path.join(DIR_ROOT_SAVE, 'flip_cifar10', 'model', 'other.pt')
PATH_KMEANS = os.path.join(DIR_ROOT_SAVE, 'k_means', 'cifar10.pt')
PATH_LOG = os.path.join(DIR_ROOT_LOG, NOTEBOOK_NAME, 'test_acc.pt')

print('train on:', ModelManager.DEVICE)

train_idx = np.arange(NUM_TRAIN, dtype=int)
test_idx = np.arange(NUM_TEST, dtype=int)
dataset_train, dataset_test, dataset_train_for_test, dataset_train_raw = get_cifar10()
# loader_train = get_loader(dataset_train, train_idx, BATCH_SIZE, shuffle=True)
loader_test = get_loader(dataset_test, test_idx, BATCH_SIZE, shuffle=False)
loader_train_ordered = get_loader(dataset_train_for_test, train_idx, BATCH_SIZE, shuffle=False)
Y_train = Tensor(dataset_train.targets)[train_idx].type(torch.int64)
Y_test = Tensor(dataset_test.targets)[test_idx].type(torch.int64)

# get prune scores
data = torch.load(PATH_EL2N)

ensemble_softmax = data['ensemble_softmax']
ensemble_pred = data['ensemble_pred']
ensemble_pred_sum = data['ensemble_pred_sum']
ensemble_std = data['ensemble_std']
el2n_scores = data['el2n_scores']
change_counter = torch.load(PATH_FORGET)['change_counter']
forget = torch.load(PATH_FORGET)['true_forget']
km = np.load(os.path.join(DIR_ROOT_SAVE, 'swav_cifar10_512', 'km_dist.npy'))

# forgetting_model = ModelManager(NUM_CLASSES, 'forgetting', load=True)
# change_counter = forgetting_model.data_other['change_counter']
#
# ensemble_softmax = torch.arange(NUM_TRAIN)
# ensemble_pred = torch.arange(NUM_TRAIN)
# ensemble_pred_sum = torch.arange(NUM_TRAIN)
# # ensemble_std =  torch.arange(NUM_TRAIN)
# el2n_scores = torch.arange(NUM_TRAIN)
# change_counter = torch.arange(NUM_TRAIN)

idx_sorted_el2n = el2n_scores.sort()[1].numpy()
idx_sorted_flip = change_counter.sort()[1].numpy()
idx_sorted_std = ensemble_std.sum(dim=1).sort()[1].numpy()
idx_sorted_pred_sum = ensemble_pred_sum.sort()[1].numpy()[::-1]
idx_sorted_forget = forget.sort()[1].numpy()
idx_random = np.arange(NUM_TRAIN)
np.random.shuffle(idx_random)
idx_sorted_km = np.argsort(km)

idx_sorted = {'random': idx_random, 'el2n': idx_sorted_el2n, 'std': idx_sorted_std,
              'pred_sum': idx_sorted_pred_sum,
              'flip': idx_sorted_flip, 'forget': idx_sorted_forget, 'km': idx_sorted_km}

data = pd.DataFrame({
    'EL2N': el2n_scores.numpy(),
    'Std': ensemble_std.sum(dim=1).numpy(),
    'True p sum': ensemble_pred_sum.numpy(),  # number of models that right on each example
    'flip': change_counter.numpy(),
    'forget': forget.numpy(),
    'km': km
})
data.corr()

train on: cuda
Files already downloaded and verified
Files already downloaded and verified


,EL2N,Std,True p sum,flip,forget,km
EL2N,1.000000,0.780107,-0.944299,0.658627,0.399751,0.529046
Std,0.780107,1.000000,-0.660269,0.586532,0.377763,0.496631
True p sum,-0.944299,-0.660269,1.000000,-0.627331,-0.350162,-0.460980
flip,0.658627,0.586532,-0.627331,1.000000,0.770378,0.394874
forget,0.399751,0.377763,-0.350162,0.770378,1.000000,0.253914
km,0.529046,0.496631,-0.460980,0.394874,0.253914,1.000000


In [2]:
no_prune = ModelManager(NUM_CLASSES, model_name='no_prune', dir_=NOTEBOOK_NAME, load=True)
# loader_train = get_loader(dataset_train, train_idx, BATCH_SIZE, True)
# no_prune.train(loader_train, loader_test, loader_test, EPOCHS)

exp_num = 0
done_0 = False

print()
for name, idx in idx_sorted.items():
    model_manager = ModelManager(NUM_CLASSES, model_name=f'{name}_exp{exp_num}', dir_=NOTEBOOK_NAME,
                                 load=done_0)

    for epochs, presents in [(10, .5), (5, .6), (5, .7), (5, .8), (5, .9), (40, 1.)]:
        print(f'====     train model with {presents}% according to {name} method    ======')
        if not done_0:
            loader_train = get_loader(dataset_train, idx[:round(presents * NUM_TRAIN)], batch_size=BATCH_SIZE,
                                      shuffle=True)
            model_manager.train(loader_train, loader_test, loader_test, epochs)
        print()


====     train model with 0.5% according to random method    ======
Epoch: 0 Training: Loss: 0.065965 Acc: 0.391920  Validation Loss: 0.050256 Acc: 0.538700                                                     
Validation loss decreased (inf --> 0.050256).  Saving model to models_data/tuning_cifar10\random_exp0
Epoch: 1 Training: Loss: 0.050944 Acc: 0.539160  Validation Loss: 0.040974 Acc: 0.634100                                                     
Validation loss decreased (0.050256 --> 0.040974).  Saving model to models_data/tuning_cifar10\random_exp0
Epoch: 2 Training: Loss: 0.043867 Acc: 0.608720  Validation Loss: 0.037081 Acc: 0.669400                                                      
Validation loss decreased (0.040974 --> 0.037081).  Saving model to models_data/tuning_cifar10\random_exp0
Epoch: 3 Training: Loss: 0.038538 Acc: 0.658120  Validation Loss: 0.030301 Acc: 0.737400                                                      
Validation loss decreased (0.037081 --> 0.030

In [3]:
exp_num = 1
done_1 = False
print()
for name, idx in idx_sorted.items():
    model_manager = ModelManager(NUM_CLASSES, model_name=f'{name}_exp{exp_num}', dir_=NOTEBOOK_NAME,
                                 load=done_1)
    for epochs, presents in [(20, .5), (10, .6), (10, .7), (10, .8), (10, .9), (20, 1.)]:
        print(f'====     train model with {presents}% according to {name} method    ======')
        if not done_1:
            loader_train = get_loader(dataset_train, idx[:round(presents * NUM_TRAIN)], batch_size=BATCH_SIZE,
                                      shuffle=True)
            model_manager.train(loader_train, loader_test, loader_test, epochs)
        print()


====     train model with 0.5% according to random method    ======
Epoch: 0 Training: Loss: 0.064911 Acc: 0.407960  Validation Loss: 0.053878 Acc: 0.531800                                                     
Validation loss decreased (inf --> 0.053878).  Saving model to models_data/tuning_cifar10\random_exp1
Epoch: 1 Training: Loss: 0.050294 Acc: 0.542960  Validation Loss: 0.042982 Acc: 0.611500                                                     
Validation loss decreased (0.053878 --> 0.042982).  Saving model to models_data/tuning_cifar10\random_exp1
Epoch: 2 Training: Loss: 0.043346 Acc: 0.611960  Validation Loss: 0.034917 Acc: 0.689300                                                      
Validation loss decreased (0.042982 --> 0.034917).  Saving model to models_data/tuning_cifar10\random_exp1
Epoch: 3 Training: Loss: 0.038559 Acc: 0.658640  Validation Loss: 0.031759 Acc: 0.720200                                                      
Validation loss decreased (0.034917 --> 0.031

In [4]:
exp_num = 2
done_2 = False
print()
for name, idx in idx_sorted.items():
    model_manager = ModelManager(NUM_CLASSES, model_name=f'{name}_exp{exp_num}', dir_=NOTEBOOK_NAME,
                                 load=done_2)
    model_manager.scheduler = torch.optim.lr_scheduler.MultiStepLR(model_manager.optimizer,
                                                                   milestones=[10, 15, 20, 25, 30], gamma=0.5,
                                                                   verbose=True)

    for epochs, presents in [(10, .5), (5, .6), (5, .7), (5, .8), (5, .9), (40, 1.)]:
        print(f'====     train model with {presents}% according to {name} method    ======')
        if not done_2:
            loader_train = get_loader(dataset_train, idx[:round(presents * NUM_TRAIN)], batch_size=BATCH_SIZE,
                                      shuffle=True)
            model_manager.train(loader_train, loader_test, loader_test, epochs)
        print()


Adjusting learning rate of group 0 to 1.0000e-03.
====     train model with 0.5% according to random method    ======
Adjusting learning rate of group 0 to 1.0000e-03.                                                                                          
Epoch: 0 Training: Loss: 0.066236 Acc: 0.393320  Validation Loss: 0.053492 Acc: 0.518800                                                     
Validation loss decreased (inf --> 0.053492).  Saving model to models_data/tuning_cifar10\random_exp2
Adjusting learning rate of group 0 to 1.0000e-03.                                                                                          
Epoch: 1 Training: Loss: 0.051206 Acc: 0.536200  Validation Loss: 0.041799 Acc: 0.621800                                                     
Validation loss decreased (0.053492 --> 0.041799).  Saving model to models_data/tuning_cifar10\random_exp2
Adjusting learning rate of group 0 to 1.0000e-03.                                                           

In [5]:
exp_num = 3
done_3 = False
print()
for name, idx in idx_sorted.items():
    model_manager = ModelManager(NUM_CLASSES, model_name=f'{name}_exp{exp_num}', dir_=NOTEBOOK_NAME,
                                 load=done_3)
    for epochs, presents in [(10, .5), (10, .6), (10, .7), (10, .8), (10, .9), (20, 1.)]:
        print(f'====     train model with {presents}% according to {name} method    ======')
        if not done_3:
            loader_train = get_loader(dataset_train, idx[:round(presents * NUM_TRAIN)], batch_size=BATCH_SIZE,
                                      shuffle=True)
            model_manager.train(loader_train, loader_test, loader_test, epochs)
        print()


====     train model with 0.5% according to random method    ======
Epoch: 0 Training: Loss: 0.065477 Acc: 0.402240  Validation Loss: 0.053583 Acc: 0.511300                                                     
Validation loss decreased (inf --> 0.053583).  Saving model to models_data/tuning_cifar10\random_exp3
Epoch: 1 Training: Loss: 0.050653 Acc: 0.548440  Validation Loss: 0.041810 Acc: 0.632700                                                     
Validation loss decreased (0.053583 --> 0.041810).  Saving model to models_data/tuning_cifar10\random_exp3
Epoch: 2 Training: Loss: 0.043061 Acc: 0.616160  Validation Loss: 0.037982 Acc: 0.662600                                                      
Validation loss decreased (0.041810 --> 0.037982).  Saving model to models_data/tuning_cifar10\random_exp3
Epoch: 3 Training: Loss: 0.038121 Acc: 0.663000  Validation Loss: 0.030046 Acc: 0.736800                                                      
Validation loss decreased (0.037982 --> 0.030

In [6]:
exp_num = 4
done_4 = False
print()
for name, idx in idx_sorted.items():
    model_manager = ModelManager(NUM_CLASSES, model_name=f'{name}_exp{exp_num}', dir_=NOTEBOOK_NAME,
                                 load=done_4)
    model_manager.scheduler = torch.optim.lr_scheduler.MultiStepLR(model_manager.optimizer,
                                                                   milestones=[10, 20, 30, 40, 50, 60, 70], gamma=0.5,
                                                                   verbose=True)

    for epochs, presents in [(10, .5), (10, .6), (10, .7), (10, .8), (10, .9), (20, 1.)]:
        print(f'====     train model with {presents}% according to {name} method    ======')
        if not done_4:
            loader_train = get_loader(dataset_train, idx[:round(presents * NUM_TRAIN)], batch_size=BATCH_SIZE,
                                      shuffle=True)
            model_manager.train(loader_train, loader_test, loader_test, epochs)
        print()


Adjusting learning rate of group 0 to 1.0000e-03.
====     train model with 0.5% according to random method    ======
Adjusting learning rate of group 0 to 1.0000e-03.                                                                                          
Epoch: 0 Training: Loss: 0.064725 Acc: 0.408760  Validation Loss: 0.048128 Acc: 0.555100                                                     
Validation loss decreased (inf --> 0.048128).  Saving model to models_data/tuning_cifar10\random_exp4
Adjusting learning rate of group 0 to 1.0000e-03.                                                                                         
Epoch: 1 Training: Loss: 0.049964 Acc: 0.556080  Validation Loss: 0.040067 Acc: 0.635000                                                      
Validation loss decreased (0.048128 --> 0.040067).  Saving model to models_data/tuning_cifar10\random_exp4
Adjusting learning rate of group 0 to 1.0000e-03.                                                           